### Ingest pit_stops.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
v_data_source = "" #Provided by Data Factory

In [ ]:
%run configuration

In [ ]:
%run common_functions

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 3, Finished, Available)

In [2]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 4, Finished, Available)

In [3]:
pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.option("multiLine", True) \
.json(f"{bronze_folder_path}/pit_stops.json")

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 5, Finished, Available)

##### Step 2 - Rename columns and add new columns
1. Rename driverId and raceId
1. Add ingestion_date with current timestamp

In [ ]:
final_df = add_ingestion_date(pit_stops_df)

In [8]:
final_df = final_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("data_source", lit(v_data_source))
display(final_df)

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, ed00537d-804e-4eea-9814-6993dda5ff9c)

##### Step 4 - Write to output to processed container in parquet format

In [6]:
final_df.write.mode('overwrite').parquet(f"{silver_folder_path}/pit_stops")

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 8, Finished, Available)

In [7]:
df_parquet = spark.read.parquet(f"{silver_folder_path}/pit_stops")
display(df_parquet)

StatementMeta(, 3f14e46b-1292-41ec-8fef-80c0cf286e4f, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, c0266281-ec47-46d4-b716-2af8acaf5be9)